# 问题一：英文版

In [34]:
import random
def is_variable(pat):
    return pat.startswith('?') and all(s.isalpha() for s in pat[1:])

def pat_to_dict(patterns):
    return {k: ' '.join(v) if isinstance(v, list) else v for k, v in patterns}


def is_pattern_segment(pattern):
    return pattern.startswith('?*') and all(a.isalpha() for a in pattern[2:])

def subsitite(rule, parsed_rules):
    if not rule: return []
    return [parsed_rules.get(rule[0], rule[0])] + subsitite(rule[1:], parsed_rules)

fail = [True, None]

def pat_match_with_seg(pattern, saying):
    if not pattern or not saying: return []
    pat = pattern[0]
    if is_variable(pat):
        return [(pat, saying[0])] + pat_match_with_seg(pattern[1:], saying[1:])
    elif is_pattern_segment(pat):
        match, index = segment_match(pattern, saying)
        return [match] + pat_match_with_seg(pattern[1:],saying[index:])
    elif pat == saying[0]:
        return pat_match_with_seg(pattern[1:], saying[1:])
    else:
        return fail

def segment_match(pattern, saying):
    seg_pat, rest = pattern[0], pattern[1:]
    seg_pat = seg_pat.replace('?*', '?')
    if not rest: return (seg_pat, saying), len(saying)  
    for i, token in enumerate(saying):
        if rest[0] == token and is_match(rest[1:], saying[(i + 1):]):
            return (seg_pat, saying[:i]), i
    return (seg_pat, saying), len(saying) 

def is_match(rest, saying):
    if not rest and not saying:
        return True
    if not all(a.isalpha() for a in rest[0]):
        return True
    if rest[0] != saying[0]:
        return False
    return is_match(rest[1:], saying[1:])

def response(saying,rule_responses):
    for key in rule_responses.keys():
        get_pattern = pat_match_with_seg(key.split(),saying.split())
        #get_pattern = pat_match(key.split(),saying.split())
        if len(get_pattern[0][1]) != len(saying.split()):
            rs_answer = random.choice(rule_responses[key])
            ls_answer = subsitite(rs_answer.split(), pat_to_dict(get_pattern))
            return ' '.join(ls_answer)
    print("I don't know what you mean?")

# 测试

In [36]:
enrule_responses = { '?*x hello ?*y': ['How do you do', 'Please state your problem'],
    '?*x I want ?*y': ['what would it mean if you got ?y', 'Why do you want ?y', 'Suppose you got ?y soon'],
    '?*x if ?*y': ['Do you really think its likely that ?y', 'Do you wish that ?y', 'What do you think about ?y', 'Really-- if ?y'],
    '?*x no ?*y': ['why not?', 'You are being a negative', 'Are you saying \'No\' just to be negative?'],
    '?*x I was ?*y': ['Were you really', 'Perhaps I already knew you were ?y', 'Why do you tell me you were ?y now?'],
    '?*x I feel ?*y': ['Do you often feel ?y ?', 'What other feelings do you have?']}
response("today I want to go to school",enrule_responses)

'what would it mean if you got to go to school'

# 问题二 支持中文和英文

In [113]:
import random
import jieba
from collections import defaultdict
fail = [True, None]

def is_variable(pat):
    return pat.startswith('?') and all(s.isalpha() for s in pat[1:])

def pat_to_dict(patterns):
    return {k: ' '.join(v) if isinstance(v, list) else v for k, v in patterns}

def is_pattern_segment(pattern):
    return pattern.startswith('?*') and all(a.isalpha() for a in pattern[2:])

def subsitite(rule, parsed_rules):
    if not rule: return []
    return [parsed_rules.get(rule[0], rule[0])] + subsitite(rule[1:], parsed_rules)

def pat_match_with_seg(pattern, saying):
    if not pattern or not saying: return []
    pat = pattern[0]
    if is_variable(pat):
        return [(pat, saying[0])] + pat_match_with_seg(pattern[1:], saying[1:])
    elif is_pattern_segment(pat):
        match, index = segment_match(pattern, saying)
        return [match] + pat_match_with_seg(pattern[1:],saying[index:])
    elif pat == saying[0]:
        return pat_match_with_seg(pattern[1:], saying[1:])
    else:
        return fail

def segment_match(pattern, saying):
    seg_pat, rest = pattern[0], pattern[1:]
    seg_pat = seg_pat.replace('?*', '?')
    if not rest: return (seg_pat, saying), len(saying)
    for i, token in enumerate(saying):
        if rest[0] == token and is_match(rest[1:], saying[(i + 1):]):
            return (seg_pat, saying[:i]), i
    return (seg_pat, saying), len(saying)

def is_match(rest, saying):
    if not rest and not saying:
        return True
    if not all(a.isalpha() for a in rest[0]):
        return True
    if rest[0] != saying[0]:
        return False
    return is_match(rest[1:], saying[1:])

def have_chinese(check_str):
    """
    判断字符串中是否包含中文
    :param check_str: {str} 需要检测的字符串
    :return: {bool} 包含返回True， 不包含返回False
    """
    for ch in check_str:
        if u'\u4e00' <= ch <= u'\u9fff':
            return True
    return False

def get_pat(pat):
    if have_chinese(pat):#判断是否含中文
        seg = list(jieba.cut(pat))
        ls=[]
        temp = []
        for ele in seg:
            if ele == '?' and temp ==[]:
                temp.append(ele)
            elif not have_chinese(ele) and temp !=[]:
                temp.append(ele)
            elif have_chinese(ele) and temp != []:
                ls.append(''.join(temp))
                ls.append(ele)
                temp = []
            else:
                ls.append(ele)
        if temp !=[]:ls.append(''.join(temp))
        return ls
    else:#不含中文的情况
        return pat.split()

def roughly_match(pat,saying):
    '''如果saying中不存在pat中的模板次（非?*x变量词），返回False'''
    for ele in pat:
        if have_chinese(ele) and ele not in saying:
            return False
    return True

def response2(saying,rule_responses):
    for key in rule_responses.keys():
        if roughly_match(get_pat(key),get_pat(saying)):
            get_pattern = pat_match_with_seg(get_pat(key),get_pat(saying))
            #get_pattern = pat_match(key.split(),saying.split())
            if len(get_pattern[0][1]) != len(get_pat(saying)):
                random_answer = random.choice(rule_responses[key])
                parse_answer = subsitite(get_pat(random_answer), pat_to_dict(get_pattern))
                if have_chinese(parse_answer):#由于中文和英文返回的字符串拼接方式不一样，加个判断是否为中文或英文answer
                    parse_answer = ''.join(parse_answer)
                    parse_answer = parse_answer.replace(' ','')
                    return parse_answer
                else:
                    return ' '.join(parse_answer)
        else:
            continue
    return (''.join(random.choice(rule_responses['?*x'])))#如果无匹配成功的，返回rule_response中的未匹配key [“?*x]

In [115]:
rule_responses = {
    '?*x hello ?*y': ['How do you do', 'Please state your problem'],
    '?*x I want ?*y': ['what would it mean if you got ?y', 'Why do you want ?y', 'Suppose you got ?y soon'],
    '?*x if ?*y': ['Do you really think its likely that ?y', 'Do you wish that ?y', 'What do you think about ?y', 'Really-- if ?y'],
    '?*x no ?*y': ['why not?', 'You are being a negative', 'Are you saying \'No\' just to be negative?'],
    '?*x I was ?*y': ['Were you really', 'Perhaps I already knew you were ?y', 'Why do you tell me you were ?y now?'],
    '?*x I feel ?*y': ['Do you often feel ?y ?', 'What other feelings do you have?'],
    '?*x你好?*y': ['你好呀', '请告诉我你的问题'],
    '?*x我想?*y': ['你觉得?y有什么意义呢？', '为什么你想?y', '你可以想想你很快就可以?y了'],
    '?*x我想要?*y': ['?x想问你，你觉得?y有什么意义呢?', '为什么你想?y', '?x觉得... 你可以想想你很快就可以有?y了', '你看?x像?y不', '我看你就像?y'],
    '?*x喜欢?*y': ['喜欢?y的哪里？', '?y有什么好的呢？', '你想要?y吗？'],
    '?*x讨厌?*y': ['?y怎么会那么讨厌呢?', '讨厌?y的哪里？', '?y有什么不好呢？', '你不想要?y吗？'],
    '?*x AI? *y': ['你为什么要提AI的事情？', '你为什么觉得AI要解决你的问题？'],
    '?*x机器人?*y': ['你为什么要提机器人的事情？', '你为什么觉得机器人要解决你的问题？'],
    '?*x对不起?*y': ['不用道歉', '你为什么觉得你需要道歉呢?'],
    '?*x我记得?*y': ['你经常会想起这个吗？', '除了?y你还会想起什么吗？', '你为什么和我提起?y'],
    '?*x如果?*y': ['你真的觉得?y会发生吗？', '你希望?y吗?', '真的吗？如果?y的话', '关于?y你怎么想？'],
    '?*x我?*z梦见?*y':['真的吗? --- ?y', '你在醒着的时候，以前想象过?y吗？', '你以前梦见过?y吗'],
    '?*x妈妈?*y': ['你家里除了?y还有谁?', '嗯嗯，多说一点和你家里有关系的', '她对你影响很大吗？'],
    '?*x爸爸?*y': ['你家里除了?y还有谁?', '嗯嗯，多说一点和你家里有关系的', '他对你影响很大吗？', '每当你想起你爸爸的时候， 你还会想起其他的吗?'],
    '?*x我愿意?*y': ['我可以帮你?y吗？', '你可以解释一下，为什么想?y'],
    '?*x我很难过，因为?*y': ['我听到你这么说， 也很难过', '?y不应该让你这么难过的'],
    '?*x难过?*y': ['我听到你这么说， 也很难过',
                 '不应该让你这么难过的，你觉得你拥有什么，就会不难过?',
                 '你觉得事情变成什么样，你就不难过了?'],
    '?*x就像?*y': ['你觉得?x和?y有什么相似性？', '?x和?y真的有关系吗？', '怎么说？'],
    '?*x和?*y都?*z': ['你觉得?z有什么问题吗?', '?z会对你有什么影响呢?'],
    '?*x和?*y一样?*z': ['你觉得?z有什么问题吗?', '?z会对你有什么影响呢?'],
    '?*x我是?*y': ['真的吗？', '?x想告诉你，或许我早就知道你是?y', '你为什么现在才告诉我你是?y'],
    '?*x我是?*y吗': ['如果你是?y会怎么样呢？', '你觉得你是?y吗', '如果你是?y，那一位着什么?'],
    '?*x你是?*y吗':  ['你为什么会对我是不是?y感兴趣?', '那你希望我是?y吗', '你要是喜欢， 我就会是?y'],
    '?*x你是?*y' : ['为什么你觉得我是?y'],
    '?*x因为?*y' : ['?y是真正的原因吗？', '你觉得会有其他原因吗?'],
    '?*x我不能?*y': ['你或许现在就能?*y', '如果你能?*y,会怎样呢？'],
    '?*x我觉得?*y': ['你经常这样感觉吗？', '除了到这个，你还有什么其他的感觉吗？'],
    '?*x我?*y你?*z': ['其实很有可能我们互相?y'],
    '?*x你为什么不?*y': ['你自己为什么不?y', '你觉得我不会?y', '等我心情好了，我就?y'],
    '?*x好的?*y': ['好的', '你是一个很正能量的人'],
    '?*x嗯嗯?*y': ['好的', '你是一个很正能量的人'],
    '?*x不嘛?*y': ['为什么不？', '你有一点负能量', '你说 不，是想表达不想的意思吗？'],
    '?*x不要?*y': ['为什么不？', '你有一点负能量', '你说 不，是想表达不想的意思吗？'],
    '?*x有些人?*y': ['具体是哪些人呢?'],
    '?*x有的人?*y': ['具体是哪些人呢?'],
    '?*x某些人?*y': ['具体是哪些人呢?'],
    '?*x每个人?*y': ['我确定不是人人都是', '你能想到一点特殊情况吗？', '例如谁？', '你看到的其实只是一小部分人'],
    '?*x所有人?*y': ['我确定不是人人都是', '你能想到一点特殊情况吗？', '例如谁？', '你看到的其实只是一小部分人'],
    '?*x总是?*y': ['你能想到一些其他情况吗?', '例如什么时候?', '你具体是说哪一次？', '真的---总是吗？'],
    '?*x一直?*y': ['你能想到一些其他情况吗?', '例如什么时候?', '你具体是说哪一次？', '真的---总是吗？'],
    '?*x或许?*y': ['你看起来不太确定'],
    '?*x可能?*y': ['你看起来不太确定'],
    '?*x他们是?*y吗？': ['你觉得他们可能不是?y？'],
    '?*x': ['很有趣', '请继续', '我不太确定我很理解你说的, 能稍微详细解释一下吗?']
}
response2('今天我觉得你很棒',rule_responses)

'其实很有可能我们互相觉得'

# 问题三


多设计一些模式，让这个程序变得更好玩，多和大家交流，看看大家有什么好玩的模式。

# 问题四

### 1.这样的程序有什么优点？有什么缺点？你有什么可以改进的方法吗？
我觉的这样的程序如果再改进一下，能解决一些日常常识型的问题，比如一些基于事实或者有点类似于检索的问答都可以这样做。
但是它的缺点也很明显，一是基于纯基于检索的问答，不灵活，如果出现没有定义在数据模板中的问句，就无法回答，所以如果想要覆盖面大，需要巨大的数据支撑。二，它是没有思想的，一切都是基于检索，并没有自己的思
对于如何改进，使用基于知识图谱的问答,看到github中挺多基于知识图谱的问答项目，虽然在问答时也是基于检索的，但是构图的过程可以看成一个学习的过程，我觉得这种方式有一定的思想；基于NLP技术的问答，通常将问句与数据库中的问句进行相似度计算，这样做会更加灵活。
对于改进的方法，
### 2.什么是数据驱动？数据驱动在这个程序里如何体现？
数据驱动，我觉得就是我们做的是从数据出发的，感觉就比如，用神经网络训练模型，是模型是从数据中学习得到，这个过程我觉得就像数据驱动
然后这个程序的体现，我认为是在回答的内容时基于你问的内容，而不是你预先定义好的。比如我喜欢苹果，根据预定义模板，可以回答你为什么喜欢苹果，但是苹果事先并没有并预定义，而是来自你问的问题中，这就是数据驱动
### 3.数据驱动与 AI 的关系是什么？
数据驱动与AI的关系，就如我上面说的，AI是从数据中学习的。之前看到过一段话，put more emphasis on making data adapt to model rather than constructing a model to match the real world. （配图）原来你说的早点睡是吃了早点再睡。